In [75]:
import pandas as pd
import re
from urllib import response
import requests
from bs4 import BeautifulSoup

In [77]:
df = pd.read_csv('국성우_공공데이터_크롤링.csv', index_col = 0)

df['schema_url'] = df['variable_url'].apply(lambda x:re.sub('.do', '.json', re.sub('/data/', '/catalog/', x)))

# lambda에서 x를 가공하고, 그 다음에 또 가공한 re.sub자체를 가공한다. 여기서, x는 한 데이터 프레임에 가공할 문자열(대상)을 의미한다.
# df.replace('https://www.data.go.kr/data/', '')가 안먹히는 이유? replace 메소드 자체가 선택적 값을 찾아서 해주는것이 아닌 값 자체(전체)를 치환해주는 것이기 때문에 re.sub 메소드를 사용해야한다.

schema_list = list(df['schema_url'])

In [78]:
url = 'https://www.data.go.kr/catalog/15086313/fileData.json'
response = requests.get(url)
soup_dict = eval(response.text)
# print(soup_dict)
df_data = pd.DataFrame([soup_dict])

In [83]:
df_data

{'name': '환경부 한강홍수통제소_홍수예보_표준유역 AWS 강수량',
 'description': '한강홍수통제소에서 수집, 측정하는 우량자료를 바탕으로 재계산한 대권역, 표준유역 별 우량 데이터입니다. 대권역명, 표준유역명, 우량 등 데이터를 제공합니다.',
 'url': 'https://www.data.go.kr/data/15086313/fileData.do',
 'keywords': ['홍수,수문,AWS'],
 'license': 'https://data.go.kr/ugs/selectPortalPolicyView.do',
 'dateCreated': '2021-08-20',
 'dateModified': '2022-07-26',
 'datePublished': '2021-08-20',
 'creator': {'name': '환경부 한강홍수통제소',
  'contactPoint': {'contactType': '예보통제과',
   'telephone': '+82-025909945',
   '@type': 'ContactPoint'},
  '@type': 'Organization'},
 'distribution': [{'encodingFormat': 'CSV',
   'contentUrl': 'https://www.data.go.kr/cmm/cmm/fileDownload.do?atchFileId=FILE_000000002576483&fileDetailSn=1&insertDataPrcus=N',
   '@type': 'DataDownload'}],
 '@context': 'https://schema.org',
 '@type': 'Dataset'}

In [81]:
df_data.drop(['creator', 'distribution', '@context', '@type'], axis=1, inplace=True)

df_data['creator_name'] = soup_dict['creator']['name']
df_data['creator_contactPoint'] = soup_dict['creator']['contactPoint']['contactType']
df_data['creator_telephone'] = soup_dict['creator']['contactPoint']['telephone']
df_data['distribution_encodingFormat'] = soup_dict['distribution'][0]['encodingFormat']
df_data['distribution_contentUrl'] = soup_dict['distribution'][0]['contentUrl']

In [74]:
df_data

,name,description,url,keywords,license,dateCreated,dateModified,datePublished,creator_name,creator_contactPoint,creator_telephone,distribution_encodingFormat,distribution_contentUrl
0,환경부 한강홍수통제소_홍수예보_표준유역 AWS 강수량,"한강홍수통제소에서 수집, 측정하는 우량자료를 바탕으로 재계산한 대권역, 표준유역 별...",https://www.data.go.kr/data/15086313/fileData.do,"[홍수,수문,AWS]",https://data.go.kr/ugs/selectPortalPolicyView.do,2021-08-20,2022-07-26,2021-08-20,환경부 한강홍수통제소,예보통제과,+82-025909945,CSV,https://www.data.go.kr/cmm/cmm/fileDownload.do...


In [58]:
# 빈데이터프레임을 선언한다 df_va
# 성개코를 돌려서, 여기서 나온 데이터를 갖다박는다. df_va와 concat
# 그리고 포문안에서 다시 df_va = pd.concat([df_va,df], axis = 1)

In [59]:
for i in schema_list[:3]:
    url = i
    response = requests.get(url)
    soup_dict = eval(response.text)
    # print(soup_dict)
    df_data = pd.DataFrame([soup_dict])
    df_data.drop(['creator', 'distribution', '@context', '@type'], axis=1, inplace=True)
    
    new_df = pd.DataFrame(columns=df_data.columns)
    new_df.loc[0, 'name'] = df_data.loc[0,'name']
    new_df.loc[0, 'description'] = soup_dict['creator']['contactPoint']['contactType']
    new_df.loc[0, 'url'] = soup_dict['creator']['contactPoint']['telephone']
    new_df.loc[0, 'keywords'] = soup_dict['distribution'][0]['encodingFormat']
    new_df.loc[0, 'license'] = soup_dict['distribution'][0]['contentUrl']
    new_df.loc[0, 'dateCreated'] = soup_dict['distribution'][0]['contentUrl']
    new_df.loc[0, 'dateModified'] = soup_dict['distribution'][0]['contentUrl']
    new_df.loc[0, 'datePublished'] = soup_dict['distribution'][0]['contentUrl']
    new_df.loc[0, 'creator_name'] = soup_dict['creator']['name']
    new_df.loc[0, 'creator_contactPoint'] = soup_dict['creator']['contactPoint']['contactType']
    new_df.loc[0, 'creator_telephone'] = soup_dict['creator']['contactPoint']['telephone']
    new_df.loc[0, 'distribution_encodingFormat'] = soup_dict['distribution'][0]['encodingFormat']
    new_df.loc[0, 'distribution_contentUrl'] = soup_dict['distribution'][0]['contentUrl']
    
    df_data = pd.concat((df_data, new_df), axis=0)
    
df_data.to_csv('정형데이터_테스트.csv', encoding='utf-8-sig')

In [60]:
df_data.columns

Index(['name', 'description', 'url', 'keywords', 'license', 'dateCreated',
       'dateModified', 'datePublished', 'creator_name', 'creator_contactPoint',
       'creator_telephone', 'distribution_encodingFormat',
       'distribution_contentUrl'],
      dtype='object')

In [61]:
schema_list[:3]

['https://www.data.go.kr/catalog/15086313/fileData.json',
 'https://www.data.go.kr/catalog/15086318/fileData.json',
 'https://www.data.go.kr/catalog/15036624/fileData.json']

In [62]:
new_df = pd.DataFrame(columns=df_data.columns)

In [63]:
new_df

,name,description,url,keywords,license,dateCreated,dateModified,datePublished,creator_name,creator_contactPoint,creator_telephone,distribution_encodingFormat,distribution_contentUrl


In [64]:
type(soup_dict['creator']['name'])

str

In [65]:
new_df.loc[0, 'name'] = df_data.loc[0,'name']
new_df.loc[0, 'description'] = soup_dict['creator']['contactPoint']['contactType']
new_df.loc[0, 'url'] = soup_dict['creator']['contactPoint']['telephone']
new_df.loc[0, 'keywords'] = soup_dict['distribution'][0]['encodingFormat']
new_df.loc[0, 'license'] = soup_dict['distribution'][0]['contentUrl']
new_df.loc[0, 'dateCreated'] = soup_dict['distribution'][0]['contentUrl']
new_df.loc[0, 'dateModified'] = soup_dict['distribution'][0]['contentUrl']
new_df.loc[0, 'datePublished'] = soup_dict['distribution'][0]['contentUrl']
new_df.loc[0, 'creator_name'] = soup_dict['creator']['name']
new_df.loc[0, 'creator_contactPoint'] = soup_dict['creator']['contactPoint']['contactType']
new_df.loc[0, 'creator_telephone'] = soup_dict['creator']['contactPoint']['telephone']
new_df.loc[0, 'distribution_encodingFormat'] = soup_dict['distribution'][0]['encodingFormat']
new_df.loc[0, 'distribution_contentUrl'] = soup_dict['distribution'][0]['contentUrl']


In [66]:
new_df

,name,description,url,keywords,license,dateCreated,dateModified,datePublished,creator_name,creator_contactPoint,creator_telephone,distribution_encodingFormat,distribution_contentUrl
0,환경부 한강홍수통제소_홍수예보_표준유역 AWS 강수량,예보통제과,+82-025909945,CSV,https://www.data.go.kr/cmm/cmm/fileDownload.do...,https://www.data.go.kr/cmm/cmm/fileDownload.do...,https://www.data.go.kr/cmm/cmm/fileDownload.do...,https://www.data.go.kr/cmm/cmm/fileDownload.do...,환경부 한강홍수통제소,예보통제과,+82-025909945,CSV,https://www.data.go.kr/cmm/cmm/fileDownload.do...


In [68]:
df_data = pd.concat((df_data, new_df), axis=0)

In [69]:
df_data

,name,description,url,keywords,license,dateCreated,dateModified,datePublished,creator_name,creator_contactPoint,creator_telephone,distribution_encodingFormat,distribution_contentUrl
0,환경부 한강홍수통제소_홍수예보_표준유역 AWS 강수량,"한강홍수통제소에서 수집, 측정하는 우량자료를 바탕으로 재계산한 대권역, 표준유역 별...",https://www.data.go.kr/data/15086313/fileData.do,"[홍수,수문,AWS]",https://data.go.kr/ugs/selectPortalPolicyView.do,2021-08-20,2022-07-26,2021-08-20,환경부 한강홍수통제소,예보통제과,+82-025909945,CSV,https://www.data.go.kr/cmm/cmm/fileDownload.do...
0,환경부 한강홍수통제소_홍수예보_표준유역 AWS 강수량,"한강홍수통제소에서 수집, 측정하는 우량자료를 바탕으로 재계산한 대권역, 표준유역 별...",https://www.data.go.kr/data/15086313/fileData.do,"[홍수,수문,AWS]",https://data.go.kr/ugs/selectPortalPolicyView.do,2021-08-20,2022-07-26,2021-08-20,환경부 한강홍수통제소,예보통제과,+82-025909945,CSV,https://www.data.go.kr/cmm/cmm/fileDownload.do...
0,환경부 한강홍수통제소_홍수예보_표준유역 AWS 강수량,예보통제과,+82-025909945,CSV,https://www.data.go.kr/cmm/cmm/fileDownload.do...,https://www.data.go.kr/cmm/cmm/fileDownload.do...,https://www.data.go.kr/cmm/cmm/fileDownload.do...,https://www.data.go.kr/cmm/cmm/fileDownload.do...,환경부 한강홍수통제소,예보통제과,+82-025909945,CSV,https://www.data.go.kr/cmm/cmm/fileDownload.do...
